At a high level, the Langchain agent will:

- Fetch the available tables from the database
- Decide which tables are relevant to the question
- Fetch the schemas for the relevant tables
- Generate a query based on the question and information from the schemas
- Double-check the query for common mistakes using an LLM
- Execute the query and return the results
- Correct mistakes surfaced by the database engine until the query is successful
- Formulate a response based on the results

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [18]:
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
import requests

In [11]:
llm = init_chat_model("openai:gpt-4.1")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x129c396d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x129c3a0d0>, root_client=<openai.OpenAI object at 0x129c38410>, root_async_client=<openai.AsyncOpenAI object at 0x129c39810>, model_name='gpt-4.1', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [19]:
# Try to connect to the database with different paths
import os

# Define possible database paths
possible_db_paths = [
    # 'boiler_data_all_sheets.db',  # Current directory
    '../boiler_data_all_sheets.db',  # Parent directory
    os.path.join('..', 'boiler_data_all_sheets.db'),  # Alternative parent directory syntax
    os.path.abspath('boiler_data_all_sheets.db'),  # Absolute path current dir
    os.path.abspath(os.path.join('..', 'boiler_data_all_sheets.db'))  # Absolute path parent dir
]

# Try each path
for db_path in possible_db_paths:
    if os.path.exists(db_path):
        print(f"Found database at: {db_path}")
        try:
            # Try to connect with SQLDatabase
            db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
            tables = db.get_usable_table_names()
            
            print(f"Dialect: {db.dialect}")
            if tables:
                print("Available tables:")
                for table in tables:
                    print(f"- {table}")
                # If we found tables, break the loop
                break
            else:
                print("No tables found in this database.")
        except Exception as e:
            print(f"Error connecting to database at {db_path}: {str(e)}")
    else:
        print(f"Database file not found at: {db_path}")

Found database at: ../boiler_data_all_sheets.db
Dialect: sqlite
Available tables:
- boiler_bol2
- boiler_bolr_1
- boiler_bolr_3
- boiler_data
- monthly_stats
- weekly_stats


In [20]:
print(f'Sample output: {db.run("SELECT * FROM boiler_bolr_1 LIMIT 1;")}')

Sample output: [('2024-04-01 00:00:00.000000', 0.0, 1489, 1300.6261740742507, 188.37382592574932, 0.1922091235263967, 1300.4339649507242, 0.0, 885, 24, 0, 0, 0, 0, 10.17, 2.7, 2.2, 127.5, 119.4, 29.2, 82.52, 155.8, 101.75, 26.4375, 6.779, 29.30000000000001, 23.99, 10.17, 36.54, 3902, 41.538768388099996, 3.0232189383, 0.47961000000000004, 1.8121200000000002, 6.436282673600006, 2.45, 17.8, 5.9139146567718, 192.487, 247.2934, 34.3801, 27.9036, 33.2, 123.45, 13.486085343228199, 50, 60, 10, 6.738, 16.738, 0.0016738, 5.59, 1.86, 900, 123.45, 16.0, 0.2999940001199976, 0.022108138699871965, 5.611462062246399, 39.20112157808004, 7.811218127575447, 8.071383427489108, 4.480424526894262, 1.6279623590466428, 4.324986573687069, 0.17973944877282327, 0.007588300895165009, 1.2661548255253716, 0.42280797339825726, 1.6889627989236289, 0.1606019911360385, 0.41509092952010984, 0.5756929206561483, 0.65, 13.53535692887574, 86.46464307112426, 0.13838592600048844, 0.09953832839195971, 0.5, 85.7267188167318, 54

### Tools for database interactions

langchain-community implements some built-in tools for interacting with our SQLDatabase, including tools for listing tables, reading table schemas, and checking and running queries

In [21]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



## 2. Using a prebuilt agent

Given these tools, we can initialize a pre-built agent in a single line. To customize our agents behavior, we write a descriptive system prompt.

In [22]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt,
)

Let's run this agent on a sample query and observe its behavior:

In [23]:
question = "What was the value of Coal ConsumptionKPI on date 4-1-24? for Boiler 1? "

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What was the value of Coal ConsumptionKPI on date 4-1-24? for Boiler 1? 
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_6GXoY9jK1sABXRhZQ9zq1GeL)
 Call ID: call_6GXoY9jK1sABXRhZQ9zq1GeL
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

boiler_bol2, boiler_bolr_1, boiler_bolr_3, boiler_data, monthly_stats, weekly_stats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_4NYkULgfAA1GvTkW6utyNLS5)
 Call ID: call_4NYkULgfAA1GvTkW6utyNLS5
  Args:
    table_names: boiler_bolr_1
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE boiler_bolr_1 (
	"Date" DATETIME, 
	"LDO Consumption(kl)" FLOAT, 
	"Coal ConsumptionFeeder(MT)" BIGINT, 
	"Coal ConsumptionK

In [9]:
question = "What is the value of Difference of Coal KPI to equivalent Coal KPI at Coal ConsumptionKPI = 1296.01 for 3rd April 2024 for boiler ? "

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the value of Difference of Coal KPI to equivalent Coal KPI at Coal ConsumptionKPI = 1296.01 for 3rd April 2024 for boiler ? 
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_0UcEPi0unRDHYsVEMdlMicfi)
 Call ID: call_0UcEPi0unRDHYsVEMdlMicfi
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

boiler_bol2, boiler_bolr_1, boiler_bolr_3, boiler_data, monthly_stats, weekly_stats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_LM8iPFsuS2iZiXXYjydQ8386)
 Call ID: call_LM8iPFsuS2iZiXXYjydQ8386
  Args:
    table_names: boiler_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE boiler_data (
	"Date" DATETIME, 
	"LDO Consumption

In [29]:
question = "who is the richest person in the world?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

who is the richest person in the world?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_EnI9EGpGZCWmByGHTszFbYUc)
 Call ID: call_EnI9EGpGZCWmByGHTszFbYUc
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

boiler_bol2, boiler_bolr_1, boiler_bolr_3, boiler_data, monthly_stats, weekly_stats
================================== Ai Message ==================================

It appears that the database available does not contain information about the wealth or net worth of people in the world. If you have another database or specific dataset related to this topic, please provide more information or clarify your request!


In [17]:
question = "What is the value of CO2 APH O/L for Flue Gas temp APH O/L = 123.35 and Weighted Air I/L temp to APH = 33.2 for 1st April 2024 for boiler 2"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the value of CO2 APH O/L for Flue Gas temp APH O/L = 123.35 and Weighted Air I/L temp to APH = 33.2 for 1st April 2024 for boiler 2
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_s47dfnWztOhaDxXJuEPQODjp)
 Call ID: call_s47dfnWztOhaDxXJuEPQODjp
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

boiler_bol2, boiler_bolr_1, boiler_bolr_3, boiler_data, monthly_stats, weekly_stats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_6KNspv813hkgXPTrzqg81lcT)
 Call ID: call_6KNspv813hkgXPTrzqg81lcT
  Args:
    table_names: boiler_bol2, boiler_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE boiler_bol2 (
	"Date" DATETIME, 
	"LDO Consump

LangGraph also comes with built-in utilities for visualizing the control flow of your application

In [ ]:
question = " "

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()